In [1]:
!nvidia-smi

Sun May 10 12:34:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# import the packages
import os
import numpy as np
import matplotlib
import gc

In [3]:
# Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load the Drive path where the data is loaded.
path='/content/drive/My Drive/Colab Notebooks/Github_Repo/The_School_of_AI/TSAI/TSAI/S14'

In [0]:
# Extracting the data by 7zipper
#!7z x '{path}'/bg.7z
#!7z x '{path}'/fgbg.7z -o'/content/Dataset'
#!7z x '{path}'/fgbgmask.7z

In [0]:
# copy the utils file from the drive
#%cp -av '{path}'/Model/ '/content/'

In [0]:
# Getting the model Weights
#!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./Model/nyu.h5

In [0]:
# Getting the variables bgs, samples and filenames with their full length names.
path='Dataset/'
bgs=[]
samples=[]
filenames=[]
for root, dirs, files in os.walk('Dataset/'):
  if root==path:
    bgs=dirs
  elif len(files)==0 and root.split('/')[-1] in bgs:
    samples = samples + dirs
  elif len(root.split('/'))==3 and root.split('/')[2] in samples:
      filenames=filenames+list(map(lambda x: os.path.join(root, x), files))

In [9]:
import Model

Using TensorFlow backend.


In [0]:
# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from Model import BilinearUpSampling2D
from Model import predict, load_images, display_images, scale_up, save_each_depth_map
from matplotlib import pyplot as plt

In [11]:
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model('/content/Model/nyu.h5', custom_objects=custom_objects, compile=False)

Loading model...


In [0]:
outputnames=list(map(lambda x: x.replace('Dataset', 'Output'),filenames))

In [0]:
def dat(intname, outname, batch):
  inputs = load_images((480,480), intname)
  # Compute results
  print('Inputs Processed')
  outputs = predict(model, inputs, batch_size=batch)
  print('Model Runned')
  del inputs
  save_each_depth_map(outputs, outname, 'gray')
  print('Saved Images')
  del outputs
  gc.collect()

In [14]:
x=0
length =int(400000/200)
for i in range(200):
  filename=filenames[x:x+length]
  outputname=outputnames[x:x+length]
  dat(filename, outputname, batch=10)
  x=x+length
  print('Processing Images for', x)

Inputs Processed
Model Runned
Saved Images
Processing Images for 2000
Inputs Processed
Model Runned
Saved Images
Processing Images for 4000
Inputs Processed
Model Runned
Saved Images
Processing Images for 6000
Inputs Processed
Model Runned
Saved Images
Processing Images for 8000
Inputs Processed
Model Runned
Saved Images
Processing Images for 10000
Inputs Processed
Model Runned
Saved Images
Processing Images for 12000
Inputs Processed
Model Runned
Saved Images
Processing Images for 14000
Inputs Processed
Model Runned
Saved Images
Processing Images for 16000
Inputs Processed
Model Runned
Saved Images
Processing Images for 18000
Inputs Processed
Model Runned
Saved Images
Processing Images for 20000
Inputs Processed
Model Runned
Saved Images
Processing Images for 22000
Inputs Processed
Model Runned
Saved Images
Processing Images for 24000
Inputs Processed
Model Runned
Saved Images
Processing Images for 26000
Inputs Processed
Model Runned
Saved Images
Processing Images for 28000
Inputs Pro

In [15]:
!7z a '/content/Densedepth.7z' '/content/Output/*'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/Output/                            20M 6419 Scan  /content/Output/039/051/                                         41M 13238 Scan  /content/Output/061/021/                                          58M 19254 Scan  /content/Output/014/001/                                          74M 24870 Scan  /content/Output/066/031/                                          94M 31688 Scan  /cont

In [16]:
path='/content/drive/My Drive/Colab Notebooks/Github_Repo/The_School_of_AI/TSAI/TSAI/S14'
%cp -av '/content/Densedepth.7z' '{path}'/

'/content/Densedepth.7z' -> '/content/drive/My Drive/Colab Notebooks/Github_Repo/The_School_of_AI/TSAI/TSAI/S14/Densedepth.7z'


In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, None, None, 6 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, None, None, 6 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

# Sample Codes to run the Depth Model.

In [0]:
inputs = load_images(filename)
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
resize_inputs= scale_up((480,640), inputs)

In [0]:
# Compute results
outputs = predict(model, resize_inputs, batch_size=10)

#matplotlib problem on ubuntu terminal fix
#matplotlib.use('TkAgg')   

In [0]:
save_each_depth_map(outputs, outputname, 'gray')

In [0]:
#rm -r Output/